## Tools for building Henderson's Mixed Model Equations

Here we will see how the mixed model equations (HMME) can be built given a data set and a model string. 


In [1]:
include ("../../PedModule.jl/src/PedModule.jl")

In [2]:
;cat small.ped

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1

In [3]:
ped = PedModule.mkPed("small.ped");

In [4]:
PedModule.PedNode(0,"0","0",-1.0)

PedNode(0,"0","0",-1.0)

In [5]:
ped.idMap

Dict{Any,Any} with 5 entries:
  "O1" => PedNode(3,"S1","D1",0.0)
  "S1" => PedNode(1,"0","0",0.0)
  "O3" => PedNode(4,"S1","D1",0.0)
  "D1" => PedNode(2,"0","0",0.0)
  "O2" => PedNode(5,"S1","D1",0.0)

In [6]:
ii,jj,vv = PedModule.HAi(ped);

In [7]:
full(HAi)

LoadError: HAi not defined
while loading In[7], in expression starting on line 1

In [8]:
Ai = HAi'HAi

LoadError: HAi not defined
while loading In[8], in expression starting on line 1

In [9]:
full(Ai)

LoadError: Ai not defined
while loading In[9], in expression starting on line 1

### A small data set to test the tools

In [10]:
A = [1,1,1,1]
B = ["S1","D1","O1","O3"]
y = [100.0, 50.0, 150.0, 40.0]

4-element Array{Float64,1}:
 100.0
  50.0
 150.0
  40.0

In [11]:
using DataFrames

In [12]:
df1 = DataFrame(intercept=A,Animal=B,y=y)

,intercept,Animal,y
1,1,S1,100.0
2,1,D1,50.0
3,1,O1,150.0
4,1,O3,40.0


In [28]:
df3 = [df1[3:4,:]  DataFrame(mat = ["D1","D1"])]

,intercept,Animal,y,mat
1,1,O1,150.0,D1
2,1,O3,40.0,D1


### Data with repeated measures

In [13]:
Animal = repeat(B,inner=[3])
Age = repmat([1,2,3],4)
intercept = ones(12,1)
df2 = DataFrame(Animal=Animal,Age=Age,y=randn(12))

,Animal,Age,y
1,S1,1,1.3397719182899197
2,S1,2,-1.070404761838825
3,S1,3,-1.6339526198053924
4,D1,1,-0.022487203640099897
5,D1,2,1.264557840818519
6,D1,3,-1.4092274450757705
7,O1,1,-3.10797802222563
8,O1,2,1.153875008796903
9,O1,3,0.8378312359774857
10,O3,1,-0.17617133428443243


In [20]:
function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    for i=1:n
        x[i] = ((b[i] - A[i,:]*x)/A[i,i])[1,1] + x[i]
    end
    diff = sum((A*x-b).^2)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[i,:]*x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        println(iter," ",diff/n)
    end
    return x
end

function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end


type TermStrVal
    str::String
    value::Float64
end

type TermLvlVal
    level::String
    value::Float64
end

type ModelTerm 
    trmStr::String
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{String,1}                    # used to store the data for this term as strings
    val::Array{Float64,1}
    startPos::Int64                         # start pos in HMME
    nLevels::Int64                           
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end

type MME
    modelEquation::String
    modelTerms::Array{ModelTerm,1}
    modelTermDict::Dict{String,ModelTerm}
    lhs::Symbol
    covVec::Array{Symbol,1}
    pedTrmVec::Array{String,1}
    mmeLhs
    mmeRhs
    ped
    Gi::Array{Float64,2}
    Ai
    mmePos::Int64
end

function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end

function getTerm(trmStr)
    trm = ModelTerm(trmStr,0,[],[],[],0,0,spzeros(0,0),[])
    factorVec = split(trmStr,"*")
    trm.nFactors = length(factorVec)
    trm.factors = [symbol(strip(f)) for f in factorVec]
    return trm
end

function initMME(modelEquation::String)
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if modelEquation==""
        println("modelEquation is empty\n")
        return
    end
    lhsRhs = split(modelEquation,"=")
    lhs = symbol(strip(lhsRhs[1]))
    rhs = strip(lhsRhs[2])
    rhsVec = split(rhs,"+")    
    dict = Dict{String,ModelTerm}()
    modelTerms = [getTerm(strip(trmStr)) for trmStr in rhsVec]
    for (i,trm) = enumerate(modelTerms)
        dict[trm.trmStr] = modelTerms[i]
    end    
    return MME(modelEquation,modelTerms,dict,lhs,[],[],0,0,0,Array(Float64,1,1),0,1)
end 

function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(String,nObs)
    trm.val = Array(Float64,nObs)
    if(trm.factors[1] == :intercept)                     # from Melanie's HW
        str = fill(string(trm.factors[1]),nObs)
        val = fill(1.0,nObs)
    else
        myDf = df[trm.factors]
        if trm.factors[1] in mme.covVec
            str = fill(string(trm.factors[1]),nObs)
            val = df[trm.factors[1]]
        else
            str = [string(i) for i in df[trm.factors[1]]]
            val = fill(1.0,nObs)
        end
        for i=2:trm.nFactors
            if trm.factors[i] in mme.covVec
                str = str .* fill("*"*string(trm.factors[i]),nObs)
                val = val .* df[trm.factors[i]]
            else
                str = str .* fill("*",nObs) .* [string(j) for j in df[trm.factors[i]]]
                val = val .* fill(1.0,nObs)
            end
        end
    end
    trm.str = str
    trm.val = val
end

getFactor1(str) = [strip(i) for i in split(str,"*")][1]

function getX(trm,mme::MME)
    pedSize = 0
    nObs  = size(trm.str,1)
    if trm.trmStr in mme.pedTrmVec
        trm.names   = PedModule.getIDs(mme.ped)
        trm.nLevels = length(mme.ped.idMap)
        xj = int([mme.ped.idMap[getFactor1(i)].seqID for i in trm.str])
    else
        dict,trm.names  = mkDict(trm.str)
        trm.nLevels     = length(dict)
        xj    = int([dict[i] for i in trm.str])
    end
    xi    = 1:nObs
    xv    = trm.val
    if mme.ped!=0
        pedSize = length(mme.ped.idMap)
        if trm.trmStr in mme.pedTrmVec
            # This is to ensure the X matrix for 
            # additive effect has the correct number of columns
            ii = 1         # adding a zero to
            jj = pedSize   # the last column in row 1
            vv = [0.0]
            xi = [xi,ii]
            xj = [xj,jj]
            xv = [xv,vv]
        end
    end  
    trm.X = sparse(xi,xj,xv)
    trm.startPos = mme.mmePos
    mme.mmePos  += trm.nLevels
end

function getMME(mme::MME, df::DataFrame)
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm,mme)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y    = df[mme.lhs]
    nObs = size(y,1)
    ii = 1:nObs
    jj = fill(1,nObs)
    vv = y
    nRowsX = size(X,1)
    if nRowsX > nObs
        ii = [ii,nRowsX]
        jj = [jj,1]
        vv = [vv,0.0]
    end
    ySparse = sparse(ii,jj,vv)
    mme.mmeLhs = X'X
    mme.mmeRhs = X'ySparse
    if mme.ped != 0
        ii,jj,vv = PedModule.HAi(mme.ped)
        HAi = sparse(ii,jj,vv)
        mme.Ai = HAi'HAi
        addA(mme::MME)
    end
end

function getNames(mme)
    names = Array(String,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end  

function covList(mme::MME, covStr::String)
    covVec = split(covStr," ",false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end

function getSolJ(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
end

function getSolG(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
end

function setAsRandom(mme::MME,randomStr::String,ped::PedModule.Pedigree, G::Array{Float64,2})
    mme.pedTrmVec = split(randomStr," ",false)
    mme.ped = ped
    mme.Gi = inv(G)
    nothing
end

function addA(mme::MME)
    pedTrmVec = mme.pedTrmVec
    #pedTrm = mme.modelTermDict[mme.pedTrmVec[1]]
    for (i,trmi) = enumerate(pedTrmVec)
        pedTrmi  = mme.modelTermDict[trmi]
        startPosi  = pedTrmi.startPos
        endPosi    = startPosi + pedTrmi.nLevels - 1
        for (j,trmj) = enumerate(pedTrmVec)
            pedTrmj  = mme.modelTermDict[trmj]
            startPosj  = pedTrmj.startPos
            endPosj    = startPosj + pedTrmj.nLevels - 1       
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] = 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] + mme.Ai*mme.Gi[i,j] 
        end
    end
end

addA (generic function with 1 method)

In [15]:
mme = initMME("y = intercept + Age + Animal + Animal*Age")
covList(mme,"Age")
nothing

In [16]:
resG = getSolG(mme,df2)

1 0.3744555851752687
2 0.27427312819569577
3 0.20105494360109538
4 0.14746994172878522
5 0.10821361519572097
6 0.07943282017822632
7 0.05832045004166405
8 0.04282697363353543
9 0.03145353984707388
10 0.02310269090960419
11 0.016970149511521374
12 0.01246610740999943
13 0.009157824116191099
14 0.006727685238810084
15 0.004942512751848092
16 0.003631084662939596
17 0.002667655138683081
18 0.0019598664286142606
19 0.0014398784023325303
20 0.0010578571815511657
21 0.000777194358324855
22 0.0005709962466724781
23 0.0004195054547345931
24 0.0003082070140685843
25 0.00022643722592920597
26 0.00016636173029471878
27 0.0001222247756244086
28 8.979770307009481e-5
29 6.597377340050956e-5
30 4.847050080523514e-5
31 3.561096583023608e-5
32 2.6163150821664033e-5
33 1.922190254421005e-5
34 1.4122211856825131e-5
35 1.0375501370263748e-5
36 7.622816675574396e-6
37 5.6004363862394e-6
38 4.114606132966877e-6
39 3.022975830993839e-6
40 2.220961779186564e-6
41 1.63172699130705e-6
42 1.1988198140804486e-6
4

10x2 Array{Any,2}:
 "intercept: intercept"  -0.289454 
 "Age: Age"               0.0462261
 "Animal: S1"             2.80416  
 "Animal: D1"             1.61841  
 "Animal: O1"            -4.02346  
 "Animal: O3"            -0.768926 
 "Animal*Age: S1*Age"    -1.53131  
 "Animal*Age: D1*Age"    -0.738713 
 "Animal*Age: O1*Age"     1.92454  
 "Animal*Age: O3*Age"     0.503969 

In [17]:
full(mme.mmeLhs)

10x10 Array{Float64,2}:
 12.0  24.0  3.0  3.0  3.0  3.0   6.0   6.0   6.0   6.0
 24.0  56.0  6.0  6.0  6.0  6.0  14.0  14.0  14.0  14.0
  3.0   6.0  3.0  0.0  0.0  0.0   6.0   0.0   0.0   0.0
  3.0   6.0  0.0  3.0  0.0  0.0   0.0   6.0   0.0   0.0
  3.0   6.0  0.0  0.0  3.0  0.0   0.0   0.0   6.0   0.0
  3.0   6.0  0.0  0.0  0.0  3.0   0.0   0.0   0.0   6.0
  6.0  14.0  6.0  0.0  0.0  0.0  14.0   0.0   0.0   0.0
  6.0  14.0  0.0  6.0  0.0  0.0   0.0  14.0   0.0   0.0
  6.0  14.0  0.0  0.0  6.0  0.0   0.0   0.0  14.0   0.0
  6.0  14.0  0.0  0.0  0.0  6.0   0.0   0.0   0.0  14.0

In [31]:
mme = initMME("y = intercept + Age + Animal + Animal*Age")
covList(mme,"Age")
G = [1 0.1; 0.1 1.0]
setAsRandom(mme,"Animal Animal*Age",ped,G)
resG = getSolG(mme,df2)

1 0.034698746180040076
2 0.019329121590071604
3 0.01238946296161013
4 0.008625394088437668
5 0.006530677579087718
6 0.005305827204967425
7 0.004540930959975607
8 0.004025605048918291
9 0.003650303296983886
10 0.0033569061325273947
11 0.003113895008425256
12 0.002903739608620177
13 0.00271640504353738
14 0.002545958716582788
15 0.0023887659004200863
16 0.002242510929058275
17 0.0021056544826768063
18 0.0019771256014389257
19 0.0018561427459463833
20 0.001742107442399668
21 0.0016345397278487302
22 0.0015330382319871407
23 0.0014372550987170136
24 0.0013468800220195738
25 0.001261629973446668
26 0.0011812425323165924
27 0.0011054715203685389
28 0.0010340841216167657
29 0.0009668589640071568
30 0.0009035848252951236
31 0.0008440597438965665
32 0.0007880903916577892
33 0.0007354916149800093
34 0.0006860860830893571
35 0.0006397040034882743
36 0.0005961828786264877
37 0.0005553672870616428
38 0.0005171086784763165
39 0.00048126517593441074
40 0.000447701381396048
41 0.00041628818223542874
4

12x2 Array{Any,2}:
 "intercept: intercept"  -0.258577   
 "Age: Age"              -0.0973098  
 "Animal: S1"             0.110911   
 "Animal: D1"            -0.106148   
 "Animal: O1"            -0.487683   
 "Animal: O3"            -0.0179881  
 "Animal: O2"             0.00238147 
 "Animal*Age: S1"        -0.176201   
 "Animal*Age: D1"         0.174373   
 "Animal*Age: O1"         0.465182   
 "Animal*Age: O3"         0.27267    
 "Animal*Age: O2"        -0.000913857

In [34]:
round(full(mme.mmeLhs),1)

12x12 Array{Float64,2}:
 12.0  24.0   3.0   3.0   3.0   3.0   0.0   6.0   6.0   6.0   6.0   0.0
 24.0  56.0   6.0   6.0   6.0   6.0   0.0  14.0  14.0  14.0  14.0   0.0
  3.0   6.0   5.5   1.5  -1.0  -1.0  -1.0   5.7  -0.2   0.1   0.1   0.1
  3.0   6.0   1.5   5.5  -1.0  -1.0  -1.0  -0.2   5.7   0.1   0.1   0.1
  3.0   6.0  -1.0  -1.0   5.0   0.0   0.0   0.1   0.1   5.8   0.0   0.0
  3.0   6.0  -1.0  -1.0   0.0   5.0   0.0   0.1   0.1   0.0   5.8   0.0
  0.0   0.0  -1.0  -1.0   0.0   0.0   2.0   0.1   0.1   0.0   0.0  -0.2
  6.0  14.0   5.7  -0.2   0.1   0.1   0.1  16.5   1.5  -1.0  -1.0  -1.0
  6.0  14.0  -0.2   5.7   0.1   0.1   0.1   1.5  16.5  -1.0  -1.0  -1.0
  6.0  14.0   0.1   0.1   5.8   0.0   0.0  -1.0  -1.0  16.0   0.0   0.0
  6.0  14.0   0.1   0.1   0.0   5.8   0.0  -1.0  -1.0   0.0  16.0   0.0
  0.0   0.0   0.1   0.1   0.0   0.0  -0.2  -1.0  -1.0   0.0   0.0   2.0

In [35]:
mme = initMME("y = intercept + Animal + mat")
G = [1 0.1; 0.1 1.0]
setAsRandom(mme,"Animal mat",ped,G)
resG = getSolG(mme,df3)

1 1.1010268725756595e-6
2 4.926239632627142e-11


11x2 Array{Any,2}:
 "intercept: intercept"   95.0        
 "Animal: S1"              0.0        
 "Animal: D1"              0.0        
 "Animal: O1"             18.3333     
 "Animal: O3"            -18.3333     
 "Animal: O2"              0.0        
 "mat: S1"                 1.75859e-16
 "mat: D1"                 0.0        
 "mat: O1"                 1.83333    
 "mat: O3"                -1.83333    
 "mat: O2"                 8.79297e-17

In [36]:
round(full(mme.mmeLhs),1)

11x11 Array{Float64,2}:
 2.0   0.0   0.0   1.0   1.0   0.0   0.0   2.0   0.0   0.0   0.0
 0.0   2.5   1.5  -1.0  -1.0  -1.0  -0.3  -0.2   0.1   0.1   0.1
 0.0   1.5   2.5  -1.0  -1.0  -1.0  -0.2  -0.3   0.1   0.1   0.1
 1.0  -1.0  -1.0   3.0   0.0   0.0   0.1   1.1  -0.2   0.0   0.0
 1.0  -1.0  -1.0   0.0   3.0   0.0   0.1   1.1   0.0  -0.2   0.0
 0.0  -1.0  -1.0   0.0   0.0   2.0   0.1   0.1   0.0   0.0  -0.2
 0.0  -0.3  -0.2   0.1   0.1   0.1   2.5   1.5  -1.0  -1.0  -1.0
 2.0  -0.2  -0.3   1.1   1.1   0.1   1.5   4.5  -1.0  -1.0  -1.0
 0.0   0.1   0.1  -0.2   0.0   0.0  -1.0  -1.0   2.0   0.0   0.0
 0.0   0.1   0.1   0.0  -0.2   0.0  -1.0  -1.0   0.0   2.0   0.0
 0.0   0.1   0.1   0.0   0.0  -0.2  -1.0  -1.0   0.0   0.0   2.0